# Example: Feature-centric analogous bars
* This notebook shows an application of the <b>feature-centric analogous bars method</b> to compare barcodes built on two different point clouds `P` and `Q`.

#### Feature-centric analogous bars
* <b> Inputs </b>
    * Distance matrix among points in `P`
    * Distance matrix among points in `Q`
    * Corss-system distance matrix among `P` and `Q`.
* <b> Goal </b>
    * Given a selected bar in the Vietoris-Rips barcode `barcode(VR(P))`, find its representations in `barcode(VR(Q))`
* <b> Implementation </b>:
    * User selects a bar of interest from `barcode(VR(P))`
    * The function `run_extension_VR_to_W_bar` finds the representation of the selected bar in the Witness complex `W(P,Q)`.
    * We then apply Dowker's theorem to find the corresponding cycle in `W(Q,P)`.
    * Lastly, we run the function `run_extension_W_to_VR_bar` to find the bar extension in `barcode(VR(Q))`.
    * Note that we don't provide an individual function for feature-centric analogous bars method. 
    * All extension methds are implemented component-wise with $\mathbb{F}_2$ coefficients. We assume that all bars of barcodes have unique death times. 
* <b> Comparison to Algorithm 6 of paper </b>
    * In the paper, we run the first extension method (from VR to W) and find the collection of all cycle extensions $E(\tau, W^\bullet_{P,Q})$. We then apply Dowker's Theorem to all such cycle extensions to find the collection $E(\tau, W^\bullet_{Q,P})$. For each cycle $[\sigma] \in E(\tau, W^\bullet_{Q,P})$, we run the second extension method (from W to VR).   
    * In our implementation, we don't apply the second extension method to the entire collection $E(\tau, W^\bullet_{Q,P})$ as this can require huge computation time. Instead, we allow the user to select a specific cycle in $E(\tau, W^\bullet_{P,Q})$ via `cycle_W_PQ`. We then find its corresponding cycle `cycle_W_QP` in $W^\bullet_{Q,P}$ via Dowker's Theorem and run the second extension method on `cycle_W_QP`. 
   

#### Example data 
* `P`: Points sampled from a "confused circle" on the torus. 
* `Q`: Points sampled from a torus. Contains `P`.
* The goal is to identify which of the S1's of `Q` is encoded in `P`.

#### Contents 
1. Load points and visualize
2. Plot the four relevant barcodes
3. Run the extension method from VR to W
4. Apply Dowker's Theorem
5. Run the extension method from W to VR. 
6. Explore cycle extension & bar extension under fixed interval decompositions of `barcode(VR(Q))`.
7. Explore alternative bar extensions under all possible interval decompositions of `barcode(VR(Q))`.

In [21]:
using Revise
includet("../../../extension_method.jl")

In [22]:
using .ext
using DelimitedFiles
using HDF5
using Printf
using Eirene
using Plots
using JLD

# 1. Load points and visualize
* `P`: a "confused circle" on a torus
* `Q`: points sampled from a torus

In [104]:
data = load("data.jld")
P = data["circle"] # 3-dimensional coordinates
P_theta = data["circle_theta"]
P_phi = data["circle_phi"]
Q = data["torus"] # 3-dimensional coordinates
Q_theta = data["torus_theta"]
Q_phi = data["torus_phi"];

# print number of points 
print("number of points in P: ", size(P,1), "\n")
print("number of points in Q: ", size(Q,1))

number of points in P: 100
number of points in Q: 300

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215


Plot the points in 3D

In [88]:
plot_3D(P, Q, P_markersize = 3, Q_markersize = 2)

<!DOCTYPE html>
 
 
 Plots.jl

Plot the points on a square torus. Assume that the sides of the squares have been identified to represent a torus

In [105]:
P_2d = hcat(P_theta, P_phi)
Q_2d = hcat(Q_theta, Q_phi)

p = plot_P_Q(P_2d, Q_2d)

<!DOCTYPE html>
 
 
 Plots.jl

# 2. Plot the four relevant barcodes

In [106]:
# compute distance
D_P, D_Q, D_P_Q, D_Q_P = compute_distance(P, Q);

In [107]:
# Compute Vietoris-Rips persistence on two regions
dim = 1
VR_P = eirene(D_P, record = "all", maxdim = dim)
VR_Q = eirene(D_Q, record = "all", maxdim = dim)

# compute Witness persistence
W_P = compute_Witness_persistence(D_P_Q, maxdim = dim)
W_Q = compute_Witness_persistence(D_Q_P, maxdim = dim);

In [108]:
# plot all four barcodes
barcode_VR_P = barcode(VR_P, dim = 1)
barcode_W_P = barcode(W_P["eirene_output"], dim = 1)
barcode_W_Q = barcode(W_Q["eirene_output"], dim = 1)
barcode_VR_Q = barcode(VR_Q, dim = 1)

# plot
p1 = plot_barcode(barcode_VR_P, lw = 3, title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_W_P, lw = 3, title = "Barcode(W(P,Q))", titlefontsize = 12)
p3 = plot_barcode(barcode_W_Q, lw = 3, title = "Barcode(W(Q,P))", titlefontsize = 12)
p4 = plot_barcode(barcode_VR_Q, lw = 2, title = "Barcode(VR(Q))", titlefontsize = 12)
plot(p1, p2, p3, p4, layout = grid(4,1), size = (500, 700))

<!DOCTYPE html>
 
 
 Plots.jl

# 3. Run the extension method from VR to W

Given the selected interval in `barcode(VR(P))`, find its cycle extensions in `W(P,Q)`

In [109]:
# select bar of interest
VR_P_class = 6

6

In [110]:
# plot cycle rep of selected bar
cycle = classrep(VR_P, class = VR_P_class, dim = 1)
cycle = [sort(cycle[:,i]) for i = 1:size(cycle,2)]
plot_cycle_square_torus(P_2d, Q_2d, cycle = cycle)

<!DOCTYPE html>
 
 
 Plots.jl

In [111]:
# run extension method
extension_to_W_PQ = run_extension_VR_to_W_bar(C_VR = VR_P,
                                              D_VR = D_P,
                                              VR_bar = VR_P_class,
                                              W = W_P,
                                              D_W = D_P_Q);

### Find all cycle extensions

In [112]:
# find all cycle extensions
CE1, _ = find_CE_BE(extension_to_W_PQ);

In [113]:
param = extension_to_W_PQ["epsilon_0"]
CE1[param]

Dict{Any,Any} with 1 entry:
  0 => [[37, 84], [2, 36], [19, 84], [25, 30], [18, 30], [1, 21], [10, 12], [1,…

Select the cycle as `cycle_W_PQ`

In [114]:
cycle_W_PQ = CE1[param][0];

In [115]:
# plot selected cycle extension
plot_cycle_square_torus(P_2d, Q_2d, cycle = cycle_W_PQ)

<!DOCTYPE html>
 
 
 Plots.jl

# 4. Find corresponding cycle in `W(Q,P)` via Dowker's Theorem

In [116]:
# find corresponding cycle
cycle_W_QP = find_Dowker_cycle_correspondence(cycle_W_PQ, param, D_P_Q);

In [120]:
cycle_W_QP = ext.select_odd_count(cycle_W_QP)

12-element Array{Array{Int64,1},1}:
 [22, 40]
 [40, 127]
 [10, 149]
 [127, 149]
 [83, 295]
 [200, 295]
 [5, 44]
 [44, 172]
 [83, 172]
 [22, 66]
 [5, 66]
 [10, 200]

In [121]:
# plot
plot_cycle_square_torus(P_2d, Q_2d, cycle = cycle_W_QP, cycle_loc = "Q")

<!DOCTYPE html>
 
 
 Plots.jl

# 5. Run the extension method from `W(Q,P)` to `VR(Q)`

Find parameter at which to run the extension method.
We'll choose the parameter immediately prior to the death time of `cycle_W_QP`.

In [122]:
# choose parameter
parameter = find_cycle_death_in_Witness(cycle_W_QP, W_Q)
psi = maximum(D_Q_P[D_Q_P.< parameter])

1.4636356070702874

In [123]:
# run extension
extension_to_VR_Q = run_extension_W_to_VR(W = W_Q,
                                          W_cycle = cycle_W_QP,
                                          psi = psi,
                                          C_VR = VR_Q,
                                          D_VR = D_Q);

### Explore results of the second extension method.

In [124]:
plot_pY(extension_to_VR_Q)

<!DOCTYPE html>
 
 
 Plots.jl

In [125]:
p = return_extension_results_at_parameter(extension_to_VR_Q)

*** Parameter key, value pair *** 
key: 1 parameter: 0.589141 
key: 2 parameter: 0.607250 
key: 3 parameter: 0.616515 
key: 4 parameter: 0.617253 
key: 5 parameter: 0.641516 
key: 6 parameter: 0.641912 
key: 7 parameter: 0.657811 
key: 8 parameter: 0.660281 
key: 9 parameter: 0.671877 
key: 10 parameter: 0.673758 
key: 11 parameter: 0.674252 
key: 12 parameter: 0.689737 
key: 13 parameter: 0.697600 
key: 14 parameter: 0.707195 
key: 15 parameter: 0.712569 
key: 16 parameter: 0.761699 
key: 17 parameter: 0.763447 
key: 18 parameter: 0.835165 
key: 19 parameter: 0.864303 
key: 20 parameter: 0.882454 
key: 21 parameter: 0.883871 
key: 22 parameter: 0.907978 
key: 23 parameter: 1.009581 
key: 24 parameter: 1.037977 



Select a key for parameter 1


Selected parameter: 0.5891407606613023

Baseline bars extension at selected parameter: [81]

*** Offset bar extensions at selected parameter *** 
key: 1 offset bar extension: [26]
key: 2 offset bar extension: [60, 28]
key: 3 offset bar extension: [22]
key: 4 offset bar extension: [19]
key: 5 offset bar extension: [60]
key: 6 offset bar extension: [61]
key: 7 offset bar extension: [21]
key: 8 offset bar extension: [23]
key: 9 offset bar extension: [62]



Select keys for offset bar extensions. 
Leave blank to select none. 
To select multiple keys, separate keys with space. ex) 1 2 3 :  



Baseline bars extension at selected parameter: [81]


<!DOCTYPE html>
 
 
 Plots.jl

# 6. Explore the bar extension under fixed interval decompositions of `barcode(VR(Q))`.

* We use the function `find_CE_BE_at_param()` to find all cycle extensions and bar extensions at a specific parameter.

In [70]:
# select parameter
param = extension_to_VR_Q["nontrivial_pY"][1]
CE_param, BE_param = find_CE_BE_at_param(extension_to_VR_Q, param);

<b> plot cycle extensions to `barcode(VR(Q))`</b>

In [71]:
@printf("number of cycle extensions at parameter %.4f : %i", param, length(CE_param))

number of cycle extensions at parameter 0.5033 : 64

In [72]:
# plot a cycle extension at selected parameter
p = plot_cycle_square_torus(P_2d, Q_2d, cycle = CE_param[0], cycle_loc = "Q", title = "cycle extension",
                                P_markersize = 5, Q_markersize = 5; legend = false)

<!DOCTYPE html>
 
 
 Plots.jl

Plot the <b>bar extensions</b> at given parameter.
* Select a cycle extension 
* Find and plot the corresponding bar extensions

In [73]:
# select parameter 
@printf("number of cycle extensions at parameter %.4f : %i", param, length(CE_param))

number of cycle extensions at parameter 0.5033 : 64

In [74]:
# select cycle extension 
y= 0

# find the corresponding bar extension
be = BE_param[y]

# plot the bar extension
barcode_VR_Q = barcode(VR_Q, dim = dim)
p = plot_barcode(barcode_VR_Q, title = "selected bar extension to barcode(VR(Q))", lw = 2, selected_bars = be, epsilon= param, v_line = [param])
plot(p, size = (500, 300))

<!DOCTYPE html>
 
 
 Plots.jl

Plot a summary analogous bars plot under fixed interval decomposition of `barcode(VR(Q))`

In [75]:
# plot the summary
l = grid(4,1)
p1 = plot_barcode(barcode_VR_P, lw = 3, selected_bars = [VR_P_class], title = "Barcode(VR(P))", titlefontsize = 12)
p2 = plot_barcode(barcode_W_P , lw = 3, title = "Barcode(W(P,Q))", titlefontsize = 12)
p3 = plot_barcode(barcode_W_Q, lw = 3, title = "Barcode(W(Q,P))", titlefontsize = 12)
p4 = plot_barcode(barcode_VR_Q, lw = 2, title = "Barcode(VR(Q))", titlefontsize = 12, selected_bars = be)
plot(p1, p2, p3, p4, layout = l, size = (500, 800))


<!DOCTYPE html>
 
 
 Plots.jl

# 7. Explore alternative bar extensions under all possible interval decompositions of `barcode(VR(Q))`.

* Up to this point, the bar extension result has been obtained for some fixed interval decomposition of `barcode(VR(Q))`.
* In this section, we show how to find the bar extensions under all possible interval decompositions. Given `cycle_W_QP` (corresponds to $[\sigma] \in E(\tau, W^\bullet_{Q,P})$ in the paper), the goal is to find $S([\sigma], X^\bullet_Q)$ from Algorithm 6 of paper. We'll refer to this set as <b>alternative bar extensions</b> since these arise from alternative choices of the interval decompositions.
* There are three different methods for exploring the alternative bar extensions. The appropriate tool depends on the sizes of the barcodes of the auxiliary filtration and the target filtration. 

1. Find all alternative bar extensions for all parameters.  
    * This is recommended for data with small barcodes. 
    * This finds the full $S([\sigma], Y^{\bullet}) = \{ S^{\mathcal{D} \circ L^{-1}}_{[y]} | \ell \in p_Y, [y] \in \mathfrak{E}_{\ell}, L \in L_Y \}$ in Algorithm 3 of paper.
2. Find alternative bar extensions at specific parameters.  
    * This is recommended for data with medium size barcodes.
    * Given a parameter $\ell$, this method finds $S([\sigma], Y^{\bullet}; \ell) = \{ S^{\mathcal{D} \circ L^{-1}}_{[y]} | [y] \in \mathfrak{E}_{\ell}, L \in L_Y \} $
3. Find alternative bar extensions of a specific bar extension.
    * This is recommended for data with large size barcodes.
    * Given a selected parameter $\ell$ and cycle extension $[y] \in \mathfrak{E}_{\ell}$, this method finds $\{S^{\mathcal{D} \circ L^{-1}}_{[y]} | L \in L_Y \}$. 
    
For this example, we'll implement method 3 due to the large number of bars `barcode(VR(Q))`. Depending on the selected parameter, the function `find_alternative_bar_extension()` can lead to an out of memory error. This will happen when a user selects a parameter at which there are numerous bars in the barcode. 

For example implementations of methods 1 and 2, see notebook `EXAMPLE_EXTENSION_VR_VR.ipynb`


In [77]:
# select a parameter and bar extension
p = return_extension_results_at_parameter(extension_to_VR_Q)

*** Parameter key, value pair *** 
key: 1 parameter: 0.503339 
key: 2 parameter: 0.504804 
key: 3 parameter: 0.506528 
key: 4 parameter: 0.512250 
key: 5 parameter: 0.515321 
key: 6 parameter: 0.515468 
key: 7 parameter: 0.528820 
key: 8 parameter: 0.538958 
key: 9 parameter: 0.566245 
key: 10 parameter: 0.575327 
key: 11 parameter: 0.575715 
key: 12 parameter: 0.579081 
key: 13 parameter: 0.584674 
key: 14 parameter: 0.592276 
key: 15 parameter: 0.593724 
key: 16 parameter: 0.601462 
key: 17 parameter: 0.634686 
key: 18 parameter: 0.636922 
key: 19 parameter: 0.647863 
key: 20 parameter: 0.664888 
key: 21 parameter: 0.668602 
key: 22 parameter: 0.695169 
key: 23 parameter: 0.758626 
key: 24 parameter: 0.788257 
key: 25 parameter: 0.807325 
key: 26 parameter: 0.824226 
key: 27 parameter: 0.831881 
key: 28 parameter: 0.890985 
key: 29 parameter: 0.973402 
key: 30 parameter: 1.034366 
key: 31 parameter: 1.076984 



Select a key for parameter 1


Selected parameter: 0.5033387522327671

Baseline bars extension at selected parameter: [84]

*** Offset bar extensions at selected parameter *** 
key: 1 offset bar extension: [67]
key: 2 offset bar extension: [22]
key: 3 offset bar extension: [25]
key: 4 offset bar extension: [24]
key: 5 offset bar extension: [20]
key: 6 offset bar extension: [68]



Select keys for offset bar extensions. 
Leave blank to select none. 
To select multiple keys, separate keys with space. ex) 1 2 3 :  



Baseline bars extension at selected parameter: [84]


<!DOCTYPE html>
 
 
 Plots.jl

In [78]:
param = param = extension_to_VR_Q["nontrivial_pY"][1]
bar_ext = [83]

1-element Array{Int64,1}:
 83

In [79]:
# find alternative representations of the selected bar extension
alt_bar_ext = find_alternative_bar_extension(extension_to_VR_Q, param, bar_extension = bar_ext)

4-element Array{Array{Int64,1},1}:
 [83]
 [67, 83]
 [68, 83]
 [67, 68, 83]

In [80]:
# plot one of the alternative bar extensions

# select an alternative bar extension
alt = alt_bar_ext[2]
barcode_Y = barcode(extension_to_VR_Q["C_VR"], dim = dim)
p =plot_barcode(barcode_Y, selected_bars = alt, lw = 3,
                    epsilon = param, v_line = [param],
                    title = "alternative bar extension")
plot(p)

<!DOCTYPE html>
 
 
 Plots.jl

In [140]:
# save the four barcodes
"""
save("feature_centric_analogous_bars.jld", 
    "barcode_VR_P", barcode_VR_P,
    "barcode_W_P", barcode_W_P,
    "barcode_W_Q", barcode_W_Q,
    "barcode_VR_Q", barcode_VR_Q,
    "bar_VR_P", [6],
    "bar_VR_Q", [81],
    "VR_P_cyclerep", hcat(cycle...), 
    "W_PQ_cyclerep", hcat(cycle_W_PQ...),
    "W_QP_cyclerep", hcat(cycle_W_QP...),
    "VR_Q_cyclerep", VR_Q_cyclerep)
"""

┌ Warning: JLD incorrectly extends FileIO functions (see FileIO documentation)
└ @ FileIO /opt/julia/packages/FileIO/JA3Vl/src/loadsave.jl:215
